checklist:


Drops
- address1 drop
- exception: clean
- new_value: dropped it (doesn't contain anything)
- current_value: dropped it (doesn't contain anything
- tax_type: clean (create a data dictionary for this column) 
- instrument_no
- sub_neighborhood': A, B, C...huh? 
- 'tax_class: all the same (residential)
- homestead: I dummied this
- building_area: nulls
- triennial_group: no cleaning necessary (float)
- address: I parsed this
- instrument_no: no clue
- tax_class2: 1 for residential
- building_area: empty (for now. check again once you get the full dataset)

cleaned
- assessor: cleaned (titlecase)
- tax_class2: cleaned and filtered
- use_code: clean (create a data dictionary for this column)
- neighborhood: clean
- owner_name: cleaned (converted to title case)
- address: cleaned (spend an hour on this!
- sale_price: cleaned (dropped commas and dollar signs)
- land_area: clean (dropped commas)
- ward: float status. cleaning not needed
- land_2017: cleaned (dropped commas and dollar signs)
- land_2018: cleaned (dropped commas and dollar signs)
- improvements_2017: cleaned (dropped commas and dollar signs)
- improvements_2018: cleaned (dropped commas and dollar signs)
- value_2017: cleaned (dropped commas and dollar signs)
- value_2018: cleaned (dropped commas and dollar signs)
- assessment_2017: cleaned (dropped commas and dollar signs)
- assessment_2018: cleaned (dropped commas and dollar signs)

filters:
- drop duplicates
- tax_class2: cleaned and filtered
- tax_class2 = 1
- sale_price > 100
- zip_code: need to filter to ^2\d+ zip codes
- city: non-dc values (look at city column)
- code: 1, only residential

Columns created
- homestead: dummified and column cleaned [drop for arima]
- zip_code: created [need to filter]
- address_1 column
- qtr
- month
- year

I need to:
- consider dropping land_area < '4'
- look at home values that are \$1
- ### add lats and longs
- eda (sns plot)
- search for 'Not Available'

I can if I have time:
- use_code: create a data dictionary for this column) [super important]
- tax_class2: create a data dictionary for this column)
- look into subneighborhood: #wtf is this?


In [14]:
import numpy as np
import pandas as pd
import math

In [15]:
path = 'otr copy3.csv'
df = pd.read_csv(path, parse_dates=['date'], infer_datetime_format=True)

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
#df = df.address.notnull() #http://bit.ly/2zk56jD

In [17]:
df.replace('', np.nan, inplace=True) #I KNOW that there are fucking empty cells. gotta fill them in.

In [18]:
df.dropna(inplace=True)

In [19]:
df['neighborhood'] = df['neighborhood'].str.title()
df['neighborhood'] = df['neighborhood'].str.replace('American Univ. Park','American University Park')
df['neighborhood'] = df['neighborhood'].str.replace('N. Cleveland Park','North Cleveland Park')

In [20]:
df['neighborhood'].unique() #this is not exhustive, but at least it's clean now

array(['American University Park', 'Anacostia', 'Barry Farms', 'Berkley',
       'Bolling Afb & Naval Res', 'Brentwood', 'Brightwood', 'Brookland',
       'Burleith', 'Capitol Hill', 'Central', 'Chevy Chase', 'Chillum',
       'Cleveland Park', 'Colonial Village', 'Columbia Heights',
       'Congress Heights', 'Crestwood', 'D.C. Village', 'Deanwood',
       'Eckington', 'Foggy Bottom', 'Forest Hills', 'Fort Dupont Park',
       'Fort Lincoln', 'Foxhall', 'Garfield', 'Georgetown', 'Glover Park',
       'Hawthorne', 'Hillcrest', 'Kalorama', 'Kent', 'Ledroit Park',
       'Lily Ponds', 'Marshall Heights', 'Mass. Ave. Heights',
       'Michigan Park', 'Mount Pleasant', 'N. Anacostia Park',
       'North Cleveland Park', 'Observatory Circle', 'Old City I'], dtype=object)

In [21]:
df['use_code2'] = df['use_code'].str.extract('(^\d{1,})', expand = True)

In [22]:
df['use_code2']=df['use_code2'].str.strip()

In [23]:
df.drop(['use_code'], 1, inplace = True)

In [24]:
df['tax_type'] = df['tax_type'].str.replace(u'\xa0', u' ')
#\xa0 is  non-breaking space in Latin1 (ISO 8859-1). replace with a space

In [25]:
df['tax_type2'] = df['tax_type'].str.extract('(^\w{2})', expand = True)

In [26]:
df.drop(['tax_type'], 1, inplace = True)

In [27]:
df['tax_type2']=df['tax_type2'].str.strip()

In [28]:
df['tax_class'] = df['tax_class'].str.replace(u'\xa0', u' ')

In [29]:
df['tax_class'].unique()

array(['001 - Residential              ',
       '002 - Commercial               ',
       '003 - Vacant                   ',
       '004 - Blighted                 ',
       '000 - Non-Taxable              ', ' '], dtype=object)

In [30]:
df['tax_class2'] = df['tax_class'].str.extract(r'([0-9]\b)', expand = True)

In [31]:
df['tax_class2'].value_counts() # we just care about the category 1: residential

1    66085
2     2679
3      245
4       92
0       10
Name: tax_class2, dtype: int64

In [32]:
df = df[df.tax_class2 == '1']
# here is where I filter only for ones (residential tax class)

In [33]:
df['tax_class2'].value_counts()

1    66085
Name: tax_class2, dtype: int64

In [34]:
df.homestead.value_counts()

** Currently receiving the Homestead Deduction*.                     33745
** Not receiving the Homestead Deduction                             28587
** Currently receiving the Senior Citizen Homestead Deduction*.       3753
Name: homestead, dtype: int64

In [35]:
homestead1 = pd.get_dummies(df.homestead).iloc[:, :]
homestead1.columns

Index(['** Currently receiving the Homestead Deduction*.  ',
       '** Currently receiving the Senior Citizen Homestead Deduction*.  ',
       '** Not receiving the Homestead Deduction '],
      dtype='object')

In [36]:
homestead1.columns = ['homestead_yes', 'homestead_senior', 'homestead_no']
df = pd.concat([df, homestead1], axis=1)

In [37]:
# consider dropping one of the homestead categories, or all three. for arima it's not necessary

In [38]:
df['assessor'] = df['assessor'].str.title()

In [39]:
df['land_area'] = df['land_area'].str.replace(',', '')

In [40]:
## consider dropping land_area < '4'

In [41]:
df['owner_name'] = df['owner_name'].str.title()

In [42]:
df['sale_price'] = df['sale_price'].str.replace(',', '').str.replace('$', '')

In [43]:
df.drop(['current_value','new_value'], axis=1, inplace=True) # we don't need these

In [44]:
df['land_2017']= df['land_2017'].str.replace(',', '').str.replace('$', '')

In [45]:
df['land_2018']= df['land_2018'].str.replace(',', '').str.replace('$', '')

In [46]:
df['improvements_2017']= df['improvements_2017'].str.replace(',', '').str.replace('$', '')

In [47]:
df['improvements_2018']= df['improvements_2018'].str.replace(',', '').str.replace('$', '')

In [48]:
df['value_2017']= df['value_2017'].str.replace(',', '').str.replace('$', '')

In [49]:
df['value_2018']= df['value_2018'].str.replace(',', '').str.replace('$', '')

In [50]:
df['assessment_2017']= df['assessment_2017'].str.replace(',', '').str.replace('$', '')

In [51]:
df['assessment_2018']= df['assessment_2018'].str.replace(',', '').str.replace('$', '')

In [ ]:
# the address column is the mailing address, not the destination. UGHHH

In [52]:
df['address'] = df['address'].str.replace('                  ', ', ')

In [53]:
df['address'] = df['address'].str.replace('                 ', ', ')

In [54]:
df['address'] = df['address'].str.replace('    ', ' ')

In [55]:
df['address'] = df['address'].str.replace('   ', ' ')

In [56]:
df['address'] = df['address'].str.replace('  ', ' ')

In [57]:
df['address'] = df['address'].str.strip()

In [58]:
df.columns

Index(['neighborhood', 'sub_neighborhood', 'exception', 'tax_class',
       'homestead', 'assessor', 'building_area', 'ward', 'land_area',
       'triennial_group', 'owner_name', 'address', 'sale_price', 'date',
       'instrument_no', 'sales_code', 'sales_type', 'land_2017', 'land_2018',
       'improvements_2017', 'improvements_2018', 'value_2017', 'value_2018',
       'assessment_2017', 'assessment_2018', 'use_code2', 'tax_type2',
       'tax_class2', 'homestead_yes', 'homestead_senior', 'homestead_no'],
      dtype='object')

In [59]:
df['address1'] = df['address'] #this gives me something to work with

In [60]:
df['address1'] = df['address1'].str.strip() #trimming 

In [61]:
df['address1'] = df['address1'].str.replace(',', '')

In [62]:
df['address1'] = df['address1'].str.replace('-', ' ')

In [63]:
df['address1'] = df['address1'].str.replace(';', '')

In [64]:
df['address1'] = df['address1'].str.replace('  ', ' ')

In [65]:
df['address1'] = df['address1'].str.replace(u'\xa0', u' ')

In [66]:
df['zip_code'] = df['address1'].str.extract('(\d{5,})', expand = True)

In [67]:
df['state'] = df['address1'].str.extract('(\d{2,}$)', expand = True)

In [68]:
df['address1'] = df['address1'].str.replace('\d{2,}$', '')

In [69]:
df['address1'] = df['address1'].str.strip() #trimming 

In [70]:
df['address1'] = df['address1'].str.replace('\d+$', '') 

In [71]:
df['state'] = df['address1'].str.extract('(\S+$)', expand = True)

In [72]:
df['address1'] = df['address1'].str.replace('\S+$', '') 

In [73]:
df['address1'] = df['address1'].str.strip() #trimming 

In [74]:
df['city'] = df['address1'].str.extract('(\S+$)', expand = True)

In [75]:
df['address_1'] = df['address1'].str.replace('(\S+$)', '')

In [76]:
df['address_1'] = df['address_1'].str.strip() #trimming 

In [77]:
#created address_1 column

In [78]:
df.drop(['address1'], 1, inplace = True)

In [79]:
df.head()

,neighborhood,sub_neighborhood,exception,tax_class,homestead,assessor,building_area,ward,land_area,triennial_group,...,use_code2,tax_type2,tax_class2,homestead_yes,homestead_senior,homestead_no,zip_code,state,city,address_1
0,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,7500,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4933 MASSACHUSETTS AVE NW
1,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,7500,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4931 MASSACHUSETTS AVE NW
2,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,6000,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4923 MASSACHUSETTS AVE NW
3,American University Park,C,No,001 - Residential,** Not receiving the Homestead Deduction,Parker Norman,,3.0,6486,3.0,...,12,TX,1,0,0,1,20814,MD,BETHESDA,7200 WISCONSIN AVE STE 903
4,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,5296,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4410 49TH ST NW


In [81]:
#if I drop the non-dc addresses, I will keep 87% of my dataset

In [84]:
df.drop_duplicates(['date', 'address_1'], inplace = True)

In [87]:
df.shape

(42105, 35)

In [88]:
#this is a really important part. Dropping duplicates. be very careful here

In [89]:
#dropping! the smallest number
df.drop_duplicates(subset=['address', 'date'], inplace = True)

In [90]:
df.shape

(42105, 35)

In [97]:
len(df[df.state=='DC'])/df.shape[0]


0.8859042869017931

In [103]:
df['sales_type'] = df['sales_type'].str.replace(u'\xa0', u' ')
#\xa0 is  non-breaking space in Latin1 (ISO 8859-1). replace with a space

In [112]:
df['sales_type'].replace(' ', np.nan, inplace=True) #I KNOW that there are fucking empty cells. gotta fill them in.

In [123]:
df.sales_type.unique()

array(['I - IMPROVED', nan, 'V - VACANT'], dtype=object)

In [124]:
df['sale_price'].replace('Not Available', np.nan, inplace = True)

In [133]:
df.shape

(42105, 38)

In [134]:
df.dropna(subset=['sale_price'], how='any', inplace = True)

In [139]:
import datetime as dt
import datetime
df['date']= pd.to_datetime(df.date)

In [140]:
df['qtr'] = df.date.dt.quarter
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

In [155]:
df['owner_name'] = df['owner_name'].str.replace(u'\xa0', u' ')

In [157]:
df['owner_name'] = df['owner_name'].str.strip()

In [164]:
df.shape

(30282, 38)

In [182]:
df.dtypes

neighborhood                 object
sub_neighborhood             object
exception                    object
tax_class                    object
homestead                    object
assessor                     object
building_area                object
ward                        float64
land_area                    object
triennial_group             float64
owner_name                   object
address                      object
sale_price                   object
date                 datetime64[ns]
instrument_no                object
sales_code                   object
sales_type                   object
land_2017                    object
land_2018                    object
improvements_2017            object
improvements_2018            object
value_2017                   object
value_2018                   object
assessment_2017              object
assessment_2018              object
use_code2                    object
tax_type2                    object
tax_class2                  

In [183]:
# df.loc[df.sale_price <= 3, :]

In [232]:
df = df[df.state == 'DC']


In [240]:
df.tax_class2.unique()

array(['1'], dtype=object)

In [236]:
cols = ['sub_neighborhood', 'exception', 'tax_class',
       'homestead', 'building_area',
       'triennial_group', 'address',
       'instrument_no', 'tax_class2']

In [241]:
df.drop(cols, 1, inplace = True) ## very important cell here!

In [249]:
df = df[df.year >= 2000] ## very important cell here!

In [ ]:
#sales_type has NaN

In [282]:
df['improvements_2017'].replace('Not Available', np.NaN, inplace = True)
df['improvements_2018'].replace('Not Available', np.NaN, inplace = True)

In [284]:
for column in df[['land_2017','land_2018', 'improvements_2017',
                 'improvements_2018','value_2017', 'value_2018', 'assessment_2017','assessment_2018']]:         
    df[column] = df[column].astype(float) ### need to filter

In [285]:
df.describe()

,ward,land_2017,land_2018,improvements_2017,improvements_2018,value_2017,value_2018,assessment_2017,assessment_2018,homestead_yes,homestead_senior,homestead_no,qtr,month,year
count,26747.000000,2.674700e+04,2.674700e+04,2.652300e+04,2.655700e+04,2.674700e+04,2.674700e+04,2.674700e+04,2.674700e+04,26747.000000,26747.000000,26747.000000,26747.000000,26747.000000,26747.000000
mean,4.506898,2.909949e+05,3.065432e+05,3.739863e+05,3.880637e+05,6.618492e+05,6.918502e+05,5.955662e+05,6.284687e+05,0.645493,0.025274,0.329233,2.580775,6.753169,2010.585449
std,2.080194,3.881424e+05,3.976415e+05,9.406325e+05,9.853857e+05,1.279731e+06,1.327380e+06,1.279933e+06,1.327516e+06,0.478373,0.156959,0.469944,1.069908,3.274468,5.061668
min,1.000000,8.700000e+02,8.700000e+02,1.000000e+02,1.000000e+02,8.700000e+02,8.700000e+02,7.600000e+02,8.700000e+02,0.000000,0.000000,0.000000,1.000000,1.000000,2000.000000
25%,3.000000,1.210000e+05,1.262400e+05,1.874450e+05,1.957900e+05,3.557100e+05,3.719550e+05,2.911360e+05,3.119400e+05,0.000000,0.000000,0.000000,2.000000,4.000000,2007.000000
50%,5.000000,2.477800e+05,2.666900e+05,2.821900e+05,2.923700e+05,5.419900e+05,5.636000e+05,4.668700e+05,4.952970e+05,1.000000,0.000000,0.000000,3.000000,7.000000,2012.000000
75%,6.000000,4.026850e+05,4.272900e+05,4.192050e+05,4.295900e+05,7.874800e+05,8.193950e+05,7.161150e+05,7.508700e+05,1.000000,0.000000,1.000000,4.000000,10.000000,2015.000000
max,8.000000,3.472220e+07,3.541665e+07,8.285049e+07,8.285049e+07,1.105066e+08,1.105066e+08,1.105066e+08,1.105066e+08,1.000000,1.000000,1.000000,4.000000,12.000000,2017.000000


In [286]:
df.to_csv('otr_clean.csv', index = False, encoding='utf-8')